In [ ]:
import sys; sys.path.insert(0, '..')  # Makes interface.py available
from interface import PluginInstanceRedisInterface
import os

# Set up redis credentials
redis_host = os.environ.get("REDIS_HOST")
redis_port = os.environ.get("REDIS_PORT")
redis_password = os.environ.get("REDIS_PASSWORD")
redis_channel = os.environ.get("REDIS_CHANNEL")

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
plugin_instance.ping()
print('Plugin Connected')

In [ ]:
from nanome.util.enums import StreamType
from interface import PluginInstanceRedisInterface

class ColorStreamPlugin(PluginInstanceRedisInterface):

    def __init__(self, redis_host, redis_port, redis_password, redis_channel=None):
        super().__init__(redis_host, redis_port, redis_password, redis_channel)
        # RGB values of the rainbow
        self.color_index = 0
        self.roygbiv = [
            (255, 0, 0),  # Red
            (255, 127, 0),  # Orange
            (255, 255, 0),  # Yellow
            (0, 255, 0),  # Green
            (0, 0, 255),  # Blue
            (75, 0, 130),  # Indigo
            (148, 0, 211),  # Violet
        ]

    def cycle_color(self, comp_indices):
        """For all atom in selected complexes, change their color."""
        print("Getting Comp")
        comps = self.request_complexes(comp_indices)
        new_color_rgba = self.roygbiv[self.color_index]

        # Create a writing stream to set colors for every atom in the complexes.
        stream_type = StreamType.color
        atom_indices = []
        for comp in comps:
            atom_indices.extend([atom.index for atom in comp.atoms])

        print("Creating Writing Stream")
        stream = self.create_writing_stream(atom_indices, stream_type)

        # Set the color for every atom in the stream.
        stream_data = []
        for _ in atom_indices:
            stream_data.extend(new_color_rgba)
        self.color_index = (self.color_index + 1) % len(self.roygbiv)
        print("Updating Stream")
        plugin_instance.stream_update(stream.id, stream_data)

plugin_instance = ColorStreamPlugin(redis_host, redis_port, redis_password, redis_channel=redis_channel)


In [ ]:
comps = plugin_instance.request_complex_list()
comp = comps[0]
print(comp)


In [ ]:
# Changing complex color
plugin_instance.cycle_color([comp.index])